In [1]:
# Dependencies
# ----------------------------------
import pandas as pd
import time
import os
import io
import requests
import hashlib
import sqlite3 as lite
import sys
from PIL import Image
from numpy import genfromtxt
from datetime import datetime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, Session

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Date

In [3]:
import selenium
from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r'C:\Users\zinil\Desktop\Chromedriver\chromedriver.exe'
wd = webdriver.Chrome(executable_path = r'C:\Users\zinil\Desktop\Chromedriver\chromedriver.exe')

# Test the Chrome connection

In [4]:
wd.get('https://google.com')

In [5]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Great Horned Owl')

In [6]:
wd.quit()

# Scraping for images functions

### This code was adapted from code described in article "Image Scraping with Python" published on https://towardsdatascience.com/ by Fabian Bosler.
## --------------------------------------------------------------------

In [7]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content
#         print(f"image_content = {image_content}")

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
        return file_path
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [9]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        
        search_term = search_term + ' bird'
        print(f'search_term = {search_term}')

    with webdriver.Chrome(executable_path=driver_path) as wd:
        search_term = search_term + ' bird'
        print(f'search_term = {search_term}')
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        print(f'images urls = {res}')
    
    image_locations = []
    for elem in res:
        print(f'elem of URLs = {elem}')
        
        bird_image_path = persist_image(target_folder,elem)
        image_locations.append(bird_image_path)
        print(f'bird image path in local foldr = {bird_image_path}')
        print(f'image_locations = {image_locations}')
        
    return image_locations

# Add pics information to birdDB.sqlite

In [10]:
# exstract list of birds English names
database_path = "data/birdDB.sqlite"

In [11]:
# Create Engine
# engine = create_engine(f"sqlite:///{database_path}")
# conn = engine.connect() #permanent connection that we are setting up
conn = lite.connect(database_path)

In [12]:
cur = conn.cursor()

In [13]:
# con = sqlite3.connect('database.db')
# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
data_master = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
# print(cursor.fetchall())
data_master.head()

,name
0,metadata
1,recordings
2,uniqueNames


In [14]:
cur.execute("SELECT sql FROM sqlite_master WHERE name='uniqueNames';")
cur.fetchall()

[('CREATE TABLE "uniqueNames" (\n"index" INTEGER,\n  "uniqueName" TEXT,\n  "imageLocation" TEXT\n)',)]

In [15]:
cur.execute("SELECT sql FROM sqlite_master;")
cur.fetchall()

[('CREATE TABLE metadata (\n\tid INTEGER, \n\t"English Name" TEXT, \n\t"Genus" TEXT, \n\t"Location" TEXT, \n\t"Latitude" FLOAT, \n\t"Longitude" FLOAT\n)',),
 ('CREATE TABLE recordings (\n\tid INTEGER, \n\t"Recording Type" TEXT, \n\t"Quality Score" TEXT, \n\t"Author Remarks" TEXT, \n\t"MP3 Download URL" TEXT, \n\t"Recording Name" TEXT\n)',),
 ('CREATE TABLE "uniqueNames" (\n"index" INTEGER,\n  "uniqueName" TEXT,\n  "imageLocation" TEXT\n)',),
 ('CREATE INDEX "ix_uniqueNames_index"ON "uniqueNames" ("index")',)]

In [16]:
cur.execute("SELECT * FROM uniqueNames ORDER BY uniqueName;")
cur.fetchall()

[(0, 'Acorn Woodpecker', './images\\acorn_woodpecker\\1651956c1e.jpg'),
 (1, 'American Bushtit', './images\\american_bushtit\\23de54f7b6.jpg'),
 (2,
  'American Cliff Swallow',
  './images\\american_cliff_swallow\\4d50746d92.jpg'),
 (3, 'American Crow', './images\\american_crow\\4aa703f5b0.jpg'),
 (4,
  'American Grey Flycatcher',
  './images\\american_grey_flycatcher\\bc34237459.jpg'),
 (5, 'American Robin', './images\\american_robin\\ddd1ec93f3.jpg'),
 (6,
  'American White Pelican',
  './images\\american_white_pelican\\980d8366f1.jpg'),
 (7, 'American Wigeon', './images\\american_wigeon\\9338a24a8a.jpg'),
 (8, "Anna's Hummingbird", "./images\\anna's_hummingbird\\97d009eaf1.jpg"),
 (9,
  'Ash-throated Flycatcher',
  './images\\ash-throated_flycatcher\\2b63d3cdc5.jpg'),
 (10, "Baird's Sandpiper", "./images\\baird's_sandpiper\\2de03d1f08.jpg"),
 (11, 'Bald Eagle', './images\\bald_eagle\\6641578ebb.jpg'),
 (12, 'Band-tailed Pigeon', './images\\band-tailed_pigeon\\a4762cebe5.jpg'),
 (13,

In [17]:
query = "SELECT * FROM uniqueNames;"

bird_images_df = pd.read_sql_query(query,conn)
bird_images_df

,index,uniqueName,imageLocation
0,0,Acorn Woodpecker,./images\acorn_woodpecker\1651956c1e.jpg
1,1,American Bushtit,./images\american_bushtit\23de54f7b6.jpg
2,2,American Cliff Swallow,./images\american_cliff_swallow\4d50746d92.jpg
3,3,American Crow,./images\american_crow\4aa703f5b0.jpg
4,4,American Grey Flycatcher,./images\american_grey_flycatcher\bc34237459.jpg
...,...,...,...
112,112,Wilson's Snipe,./images\wilson's_snipe\e2279f5393.jpg
113,113,Wood Duck,./images\wood_duck\7173954478.jpg
114,114,Wrentit,./images\wrentit\a43f0b1268.jpg
115,115,Yellow Rail,./images\yellow_rail\d5d6b7133e.jpg


In [180]:
bird_images_df.loc[bird_images_df["uniqueName"] == 'Canada Goose', ['imageLocation']] = "prova" 

In [181]:
bird_images_df.loc[bird_images_df["uniqueName"] == 'Canada Goose'] 

,uniqueName,imageLocation
31,Canada Goose,prova


In [182]:
image_paths = search_and_download('Canada Goose', DRIVER_PATH, './images', number_images=1)
print(f"image_paths = {image_paths}")
bird_images_df.loc[bird_images_df["uniqueName"] == 'Canada Goose', ['imageLocation']] = image_paths


search_term = Canada Goose bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/h_a1_6937_6_canada_goose_ethel_oneal_adult.jpg?itok=wS8xJyfz'}
elem of URLs = https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/h_a1_6937_6_canada_goose_ethel_oneal_adult.jpg?itok=wS8xJyfz
SUCCESS - saved https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/h_a1_6937_6_canada_goose_ethel_oneal_adult.jpg?itok=wS8xJyfz - as ./images\canada_goose\81a52e0675.jpg
bird image path in local foldr = ./images\canada_goose\81a52e0675.jpg
image_locations = ['./images\\canada_goose\\81a52e0675.jpg']
image_paths = ['./images\\canada_goose\\81a52e0675.jpg']


In [183]:
bird_images_df.loc[bird_images_df["uniqueName"] == 'Canada Goose'] 

,uniqueName,imageLocation
31,Canada Goose,./images\canada_goose\81a52e0675.jpg


## Insert bird images scraped with google  into uniqueNames table

In [184]:
query = "SELECT uniqueName FROM uniqueNames;"

df = pd.read_sql_query(query,conn)

for u in df['uniqueName']:
    image_paths = search_and_download(u, DRIVER_PATH, './images', number_images=1)
    bird_images_df.loc[bird_images_df["uniqueName"] == u, ['imageLocation']] = image_paths
    
    print('------------------------------------')
    print(f'Searched the following bird {u}')
    print('------------------------------------')
    bird_images_df

search_term = Acorn Woodpecker bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/64992871-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/64992871-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/64992871-480px.jpg - as ./images\acorn_woodpecker\1651956c1e.jpg
bird image path in local foldr = ./images\acorn_woodpecker\1651956c1e.jpg
image_locations = ['./images\\acorn_woodpecker\\1651956c1e.jpg']
------------------------------------
Searched the following bird Acorn Woodpecker
------------------------------------
search_term = American Bushtit bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/68041771-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/68041771-480px.jpg
SUCCESS - saved https://www.allabout

elem of URLs = https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/web_h_p1_4047_1_bald-eagle_bonnie_block_kk-adult.jpg?itok=QgqvBtXw
SUCCESS - saved https://www.audubon.org/sites/default/files/styles/hero_cover_bird_page/public/web_h_p1_4047_1_bald-eagle_bonnie_block_kk-adult.jpg?itok=QgqvBtXw - as ./images\bald_eagle\6641578ebb.jpg
bird image path in local foldr = ./images\bald_eagle\6641578ebb.jpg
image_locations = ['./images\\bald_eagle\\6641578ebb.jpg']
------------------------------------
Searched the following bird Bald Eagle
------------------------------------
search_term = Band-tailed Pigeon bird
Found: 60 search results. Extracting links from 0:60
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/164025671-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/164025671-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/164025671-480px.jpg - as ./images\band-tai

search_term = Blue-grey Gnatcatcher bird
Found: 120 search results. Extracting links from 0:120
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/68033861-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/68033861-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/68033861-480px.jpg - as ./images\blue-grey_gnatcatcher\6207bd4c14.jpg
bird image path in local foldr = ./images\blue-grey_gnatcatcher\6207bd4c14.jpg
image_locations = ['./images\\blue-grey_gnatcatcher\\6207bd4c14.jpg']
------------------------------------
Searched the following bird Blue-grey Gnatcatcher
------------------------------------
search_term = Bohemian Waxwing bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/162296061-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/162296061-480px.jpg
SUCCESS -

SUCCESS - saved https://www.allaboutbirds.org/guide/assets/og/75303631-1200px.jpg - as ./images\cedar_waxwing\4839c1ec94.jpg
bird image path in local foldr = ./images\cedar_waxwing\4839c1ec94.jpg
image_locations = ['./images\\cedar_waxwing\\4839c1ec94.jpg']
------------------------------------
Searched the following bird Cedar Waxwing
------------------------------------
search_term = Chestnut-backed Chickadee bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/68054211-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/68054211-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/68054211-480px.jpg - as ./images\chestnut-backed_chickadee\fa178d645f.jpg
bird image path in local foldr = ./images\chestnut-backed_chickadee\fa178d645f.jpg
image_locations = ['./images\\chestnut-backed_chickadee\\fa178d645f.jpg']
---------------------------------

search_term = Great Crested Flycatcher bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/65682781-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/65682781-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/65682781-480px.jpg - as ./images\great_crested_flycatcher\4551fc5ee8.jpg
bird image path in local foldr = ./images\great_crested_flycatcher\4551fc5ee8.jpg
image_locations = ['./images\\great_crested_flycatcher\\4551fc5ee8.jpg']
------------------------------------
Searched the following bird Great Crested Flycatcher
------------------------------------
search_term = Great Horned Owl bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/63741611-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/63741611-480px.

search_term = Mountain Bluebird bird
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/67471171-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/67471171-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/67471171-480px.jpg - as ./images\mountain_bluebird\01a075b62a.jpg
bird image path in local foldr = ./images\mountain_bluebird\01a075b62a.jpg
image_locations = ['./images\\mountain_bluebird\\01a075b62a.jpg']
------------------------------------
Searched the following bird Mountain Bluebird
------------------------------------
search_term = Mountain Chickadee bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/68054751-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/68054751-480px.jpg
SUCCESS - saved https://www.a

search_term = Pectoral Sandpiper bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/37807421-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/37807421-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/37807421-480px.jpg - as ./images\pectoral_sandpiper\d6d88e42ee.jpg
bird image path in local foldr = ./images\pectoral_sandpiper\d6d88e42ee.jpg
image_locations = ['./images\\pectoral_sandpiper\\d6d88e42ee.jpg']
------------------------------------
Searched the following bird Pectoral Sandpiper
------------------------------------
search_term = Peregrine Falcon bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/160654851-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/160654851-480px.jpg
SUCCESS - saved https://

search_term = Ruby-crowned Kinglet bird
Found: 120 search results. Extracting links from 0:120
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/67474351-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/67474351-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/67474351-480px.jpg - as ./images\ruby-crowned_kinglet\4a54decf61.jpg
bird image path in local foldr = ./images\ruby-crowned_kinglet\4a54decf61.jpg
image_locations = ['./images\\ruby-crowned_kinglet\\4a54decf61.jpg']
------------------------------------
Searched the following bird Ruby-crowned Kinglet
------------------------------------
search_term = Ruddy Duck bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/63912421-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/63912421-480px.jpg
SUCCESS - saved https:

search_term = Swainson's Thrush bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/67460821-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/67460821-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/67460821-480px.jpg - as ./images\swainson's_thrush\e013a08d8a.jpg
bird image path in local foldr = ./images\swainson's_thrush\e013a08d8a.jpg
image_locations = ["./images\\swainson's_thrush\\e013a08d8a.jpg"]
------------------------------------
Searched the following bird Swainson's Thrush
------------------------------------
search_term = Townsend's Solitaire bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/67469871-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/67469871-480px.jpg
SUCCESS - saved https://www

search_term = Western Wood Pewee bird
Found: 200 search results. Extracting links from 0:200
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/65617091-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/65617091-480px.jpg
SUCCESS - saved https://www.allaboutbirds.org/guide/assets/photo/65617091-480px.jpg - as ./images\western_wood_pewee\d75eaccfcb.jpg
bird image path in local foldr = ./images\western_wood_pewee\d75eaccfcb.jpg
image_locations = ['./images\\western_wood_pewee\\d75eaccfcb.jpg']
------------------------------------
Searched the following bird Western Wood Pewee
------------------------------------
search_term = White-breasted Nuthatch bird
Found: 120 search results. Extracting links from 0:120
Found: 1 image links, done!
images urls = {'https://www.allaboutbirds.org/guide/assets/photo/68039391-480px.jpg'}
elem of URLs = https://www.allaboutbirds.org/guide/assets/photo/68039391-480px.jpg
SUCCESS - saved htt

In [185]:
bird_images_df

,uniqueName,imageLocation
0,Acorn Woodpecker,./images\acorn_woodpecker\1651956c1e.jpg
1,American Bushtit,./images\american_bushtit\23de54f7b6.jpg
2,American Cliff Swallow,./images\american_cliff_swallow\4d50746d92.jpg
3,American Crow,./images\american_crow\4aa703f5b0.jpg
4,American Grey Flycatcher,./images\american_grey_flycatcher\bc34237459.jpg
...,...,...
112,Wilson's Snipe,./images\wilson's_snipe\e2279f5393.jpg
113,Wood Duck,./images\wood_duck\7173954478.jpg
114,Wrentit,./images\wrentit\a43f0b1268.jpg
115,Yellow Rail,./images\yellow_rail\d5d6b7133e.jpg


In [187]:
bird_images_df.to_sql('uniqueNames', conn, if_exists='replace')

In [188]:
cur.execute("SELECT * FROM uniqueNames ORDER BY uniqueName;")
cur.fetchall()

[(0, 'Acorn Woodpecker', './images\\acorn_woodpecker\\1651956c1e.jpg'),
 (1, 'American Bushtit', './images\\american_bushtit\\23de54f7b6.jpg'),
 (2,
  'American Cliff Swallow',
  './images\\american_cliff_swallow\\4d50746d92.jpg'),
 (3, 'American Crow', './images\\american_crow\\4aa703f5b0.jpg'),
 (4,
  'American Grey Flycatcher',
  './images\\american_grey_flycatcher\\bc34237459.jpg'),
 (5, 'American Robin', './images\\american_robin\\ddd1ec93f3.jpg'),
 (6,
  'American White Pelican',
  './images\\american_white_pelican\\980d8366f1.jpg'),
 (7, 'American Wigeon', './images\\american_wigeon\\9338a24a8a.jpg'),
 (8, "Anna's Hummingbird", "./images\\anna's_hummingbird\\97d009eaf1.jpg"),
 (9,
  'Ash-throated Flycatcher',
  './images\\ash-throated_flycatcher\\2b63d3cdc5.jpg'),
 (10, "Baird's Sandpiper", "./images\\baird's_sandpiper\\2de03d1f08.jpg"),
 (11, 'Bald Eagle', './images\\bald_eagle\\6641578ebb.jpg'),
 (12, 'Band-tailed Pigeon', './images\\band-tailed_pigeon\\a4762cebe5.jpg'),
 (13,

In [189]:
query = "SELECT * FROM uniqueNames;"

check_df = pd.read_sql_query(query,conn)
check_df

,index,uniqueName,imageLocation
0,0,Acorn Woodpecker,./images\acorn_woodpecker\1651956c1e.jpg
1,1,American Bushtit,./images\american_bushtit\23de54f7b6.jpg
2,2,American Cliff Swallow,./images\american_cliff_swallow\4d50746d92.jpg
3,3,American Crow,./images\american_crow\4aa703f5b0.jpg
4,4,American Grey Flycatcher,./images\american_grey_flycatcher\bc34237459.jpg
...,...,...,...
112,112,Wilson's Snipe,./images\wilson's_snipe\e2279f5393.jpg
113,113,Wood Duck,./images\wood_duck\7173954478.jpg
114,114,Wrentit,./images\wrentit\a43f0b1268.jpg
115,115,Yellow Rail,./images\yellow_rail\d5d6b7133e.jpg


In [195]:
cur.execute("DROP TABLE birdUniquenames;")
cur.fetchall()

[]

In [196]:
cur.execute("SELECT sql FROM sqlite_master;")
cur.fetchall()

[('CREATE TABLE metadata (\n\tid INTEGER, \n\t"English Name" TEXT, \n\t"Genus" TEXT, \n\t"Location" TEXT, \n\t"Latitude" FLOAT, \n\t"Longitude" FLOAT\n)',),
 ('CREATE TABLE recordings (\n\tid INTEGER, \n\t"Recording Type" TEXT, \n\t"Quality Score" TEXT, \n\t"Author Remarks" TEXT, \n\t"MP3 Download URL" TEXT, \n\t"Recording Name" TEXT\n)',),
 ('CREATE TABLE "uniqueNames" (\n"index" INTEGER,\n  "uniqueName" TEXT,\n  "imageLocation" TEXT\n)',),
 ('CREATE INDEX "ix_uniqueNames_index"ON "uniqueNames" ("index")',)]